In [ ]:
import pandas as pd
import pyodbc
import os


In [3]:
# Database Connection
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=RLGOKC-DB01;"
    "DATABASE=DW;"
    "Trusted_Connection=yes;"
)


In [4]:
queries = {
    "staff_revenue": """
        SELECT Staff, sum(TimeEntryGross) as YTD_Actual from fact.TimeEntries t
        where t.[TimeEntryYear] = '2024'
        group by Staff
    """,

    "team_hours": """
        SELECT 
        MONTH(t.[TimeEntryDate]) AS Month,
        SUM(t.TimeEntryAmount) AS Total_Hours
        FROM fact.TimeEntries t
        WHERE t.TimeEntryYear = 2024
        GROUP BY MONTH(t.[TimeEntryDate])
        ORDER BY Month;
    """,
    "individual_hours": """
        SELECT 
        Staff, 
        DATEPART(WEEK, t.TimeEntryDate) AS WeekNumber, 
        DATEADD(DAY, -DATEPART(WEEKDAY, t.TimeEntryDate) + 2, t.TimeEntryDate) AS WeekStartDate,
        SUM(t.TimeEntryAmount) AS Total_Hours
        FROM fact.TimeEntries t
        WHERE t.TimeEntryYear = 2024
        GROUP BY Staff, DATEPART(WEEK, t.TimeEntryDate), 
                 DATEADD(DAY, -DATEPART(WEEKDAY, t.TimeEntryDate) + 2, t.TimeEntryDate)
        ORDER BY Staff, WeekNumber;

    """,

}

In [5]:
# Fetch Data and Save as CSV
for name, query in queries.items():
    print(f"Fetching {name} data...")
    df = pd.read_sql(query, conn)
    
    # Save the dataset in the 'data' folder
    csv_path = f"data/{name}.csv"
    df.to_csv(csv_path, index=False)
    
    print(f"Saved: {csv_path}")

# Close the database connection
conn.close()
print("All datasets created successfully!")

Fetching staff_revenue data...
Saved: data/staff_revenue.csv
Fetching team_hours data...
Saved: data/team_hours.csv
Fetching individual_hours data...
Saved: data/individual_hours.csv
All datasets created successfully!


C:\Users\v_rroberson\AppData\Local\Temp\2\ipykernel_235464\2340702716.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
